In [3]:
import pandas as pd
import numpy as np
from pathlib import Path
import sys
sys.path.append(str(Path.cwd().parent))
from utils.odds_calc_utils import bookie_yield

ModuleNotFoundError: No module named 'oddspage.utils.odds_calc_utils'; 'oddspage.utils' is not a package

In [32]:
import sys

In [38]:
sys.path.append('projects/oddspage/utils/')

In [2]:
def bookie_yield(H, D, A):
    try:
        x = (1/H + 1/D + 1/A - 1)
    except ZeroDivisionError:
        x = np.nan
    return x

In [25]:
# do not read the index column
df_odds = pd.read_csv('data/0_Odds-data.csv', index_col=0)

In [21]:
# read all csv within data/2022 folder to one pandas dataframe
df_odds_2023 = pd.concat([pd.read_csv(file, engine='python') for file in Path("data/2023").rglob("*.csv")])
df_odds_2022 = pd.concat([pd.read_csv(file, engine='python') for file in Path("data/2022").rglob("*.csv")])

In [27]:
for year in [2022,2023]:
    files = Path(f"data/{year}/").rglob("*.csv")
    df_main_csvs = [pd.read_csv(file, engine='python') for file in files]
    df_main_csvs = pd.concat(df_main_csvs)
    df_main_csvs = df_main_csvs.reset_index(drop=True)

    # Find odds for as many matches as possible:
    home = ['VCH', 'B365H', 'BWH', 'IWH', 'WHH', 'PSH']
    draw = ['VCD', 'B365D', 'BWD', 'IWD', 'WHD', 'PSD']
    away = ['VCA', 'B365A', 'BWA', 'IWA', 'WHA', 'PSA']
    df_main_csvs['mH'] = df_main_csvs.apply(lambda x: max(x[home].dropna(), default=np.nan), axis=1)
    df_main_csvs['mD'] = df_main_csvs.apply(lambda x: max(x[draw].dropna(), default=np.nan), axis=1)
    df_main_csvs['mA'] = df_main_csvs.apply(lambda x: max(x[away].dropna(), default=np.nan), axis=1)

    # Save relevant columns only:
    df_main_csvs = df_main_csvs[['HomeTeam', 'AwayTeam', 'Date', 'Div', 'FTAG', 'FTHG', 'FTR', 'mH', 'mD', 'mA']]
    # Caluclate bookie yield:
    df_main_csvs['Bookie_yield'] = df_main_csvs.apply(lambda x: bookie_yield(x.mH, x.mD, x.mA), axis=1)
    df_odds = df_odds.append(df_main_csvs)

In [12]:
pd.read_excel("data/all-euro-data-2021-2022.xlsx")["Div"].unique()

array(['E0'], dtype=object)